In [1]:
#Import
import pandas as pd
import numpy as np

import json
from pandas.io.json import json_normalize

In [2]:
# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
""" Pedictions """

' Pedictions '

In [4]:
df = pd.read_json('../Data/V10/dblp-ref/dblp-ref-0.json', lines=True)

In [5]:
df_copy = df.copy()

In [6]:
df_time = df[['year', 'n_citation']].copy()

df_time['year_citation_sum'] = df_time['n_citation']
df_time['year_citation_max'] = df_time['n_citation']
df_time['year_citation_count'] = df_time['n_citation']

df_time = df_time.drop(columns = ['n_citation'])

df_time = df_time.groupby('year').agg({'year_citation_sum' : 'sum', 'year_citation_max' : 'max', 'year_citation_count' : 'count'})

df_time['year'] = df_time.index

df_time['year_citation_avg'] = df_time['year_citation_sum']/df_time['year_citation_count']

df = pd.merge(df, df_time[['year', 'year_citation_sum', 'year_citation_max', 'year_citation_count', 'year_citation_avg']], how='left', on='year')

df['age'] = 2018-df['year']

df_venue = df[['venue', 'n_citation']].copy()

df_venue['venue_citation_sum'] = df_venue['n_citation']
df_venue['venue_citation_max'] = df_venue['n_citation']
df_venue['venue_citation_count'] = df_venue['n_citation']

df_venue = df_venue.drop(columns = ['n_citation'])

df_venue = df_venue.groupby('venue').agg({'venue_citation_sum' : 'sum', 'venue_citation_max' : 'max', 'venue_citation_count' : 'count'})

df_venue['venue'] = df_venue.index

df_venue['venue_citation_avg'] = df_venue['venue_citation_sum']/df_venue['venue_citation_count']

df = pd.merge(df, df_venue[['venue', 'venue_citation_sum', 'venue_citation_max', 'venue_citation_count', 'venue_citation_avg']], how='left', on='venue')

df_domain = df[['Topic', 'n_citation']].copy()

df_domain['domain_citation_sum'] = df_domain['n_citation']
df_domain['domain_citation_max'] = df_domain['n_citation']
df_domain['domain_citation_count'] = df_domain['n_citation']

df_domain = df_domain.drop(columns = ['n_citation'])

df_domain = df_domain.groupby('Topic').agg({'domain_citation_sum' : 'sum', 'domain_citation_max' : 'max', 'domain_citation_count' : 'count'})

df_domain['Topic'] = df_domain.index

df_domain['domain_citation_avg'] = df_domain['domain_citation_sum']/df_domain['domain_citation_count']

df = pd.merge(df, df_domain[['Topic', 'domain_citation_sum', 'domain_citation_max', 'domain_citation_count', 'domain_citation_avg']], how='left', on='Topic')

df_citation_sentiment = pd.read_pickle('../Pickle/pickle_citation_sentiment_analysis')

df = pd.merge(df, df_citation_sentiment[['id', 'citation_sentiment']], how='inner', on='id')

df['time_factor'] = (df['n_citation'])/(df['year_citation_sum']*df['year_citation_max'])

df_reach_value = pd.read_pickle('../Pickle/reach_df_join.pkl')

df = pd.merge(df, df_reach_value[['id', 'reach']], how='inner', on='id')



/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'year' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'venue' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'Topic' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df.dtypes

abstract                  object
authors                   object
id                        object
n_citation                 int64
references                object
title                     object
venue                      int64
year                       int64
importance                 int64
author_count               int64
references_count         float64
Topic                      int64
pageRankValue            float64
totalOutbounds           float64
ratio                    float64
sentiment                  int64
year_citation_sum          int64
year_citation_max          int64
year_citation_count        int64
year_citation_avg        float64
age                        int64
venue_citation_sum         int64
venue_citation_max         int64
venue_citation_count       int64
venue_citation_avg       float64
domain_citation_sum        int64
domain_citation_max        int64
domain_citation_count      int64
domain_citation_avg      float64
citation_sentiment       float64
time_facto

In [9]:
df_prediction = df[['id', 'authors', 'author_count', 'sentiment', 'Topic', 'domain_citation_avg', 'year', 'time_factor']]


In [10]:
df_prediction.head()

,id,authors,author_count,sentiment,Topic,domain_citation_avg,year,time_factor
0,001c8744-73c4-4b04-9364-22d31a10dbf1,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",4,1,1,57.087180,2009,8.399536e-09
1,00bcf2d5-1592-46b0-81fd-933f90b5ecca,"[Alvaro L. Islas, Constance M. Schober]",2,0,8,54.804301,2002,5.500736e-09
2,00c85316-bddf-4bcb-93f5-097adadd73c2,"[Patrick Cousot, Radhia Cousot]",2,1,2,55.424089,1991,6.520059e-08
3,010d4ce9-0279-4166-ae73-14551ded6404,"[John R. Douceur, Jeremy Elson, Jon Howell, Ja...",4,1,9,52.136057,2008,4.063689e-09
4,012b88ae-a763-45d6-8f19-2ec9ff739d5f,"[Carmen Fernández-Daza Álvarez, Philippe Langl...",3,-1,5,55.809836,2004,6.345459e-09


In [11]:
df_paper_ranking_zscore = pd.read_pickle('ayush_ranking_score_plus_z_score')

In [12]:
df_prediction = pd.merge(df_prediction, df_paper_ranking_zscore[['id', 'ranking_score']], on='id', how='inner')

In [13]:
df_prediction.head()

,id,authors,author_count,sentiment,Topic,domain_citation_avg,year,time_factor,ranking_score
0,001c8744-73c4-4b04-9364-22d31a10dbf1,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",4,1,1,57.087180,2009,8.399536e-09,-1.309793
1,00bcf2d5-1592-46b0-81fd-933f90b5ecca,"[Alvaro L. Islas, Constance M. Schober]",2,0,8,54.804301,2002,5.500736e-09,-1.254513
2,00c85316-bddf-4bcb-93f5-097adadd73c2,"[Patrick Cousot, Radhia Cousot]",2,1,2,55.424089,1991,6.520059e-08,-2.215803
3,010d4ce9-0279-4166-ae73-14551ded6404,"[John R. Douceur, Jeremy Elson, Jon Howell, Ja...",4,1,9,52.136057,2008,4.063689e-09,16.544018
4,012b88ae-a763-45d6-8f19-2ec9ff739d5f,"[Carmen Fernández-Daza Álvarez, Philippe Langl...",3,-1,5,55.809836,2004,6.345459e-09,-1.691160


In [14]:
df_author_score = pd.read_pickle('prediction_author_score')

In [15]:
df_author_score.head()

,n_citation_sum,n_citation_h_index_utility,ranking_score_sum,pageRankValue_sum,Topic_first,author_name
authors,,,,,,
(TYPE=name) (SCHEME=Vancouver) Kahn Ce,183,4,-8.008489,3.537809,2,(TYPE=name) (SCHEME=Vancouver) Kahn Ce
-Jr. Paulo Drews,19,1,-0.952377,0.000000,5,-Jr. Paulo Drews
344ehůřek,50,1,-1.167167,0.000000,5,344ehůřek
A Aart Blokhuis,232,3,-2.580458,60.311448,8,A Aart Blokhuis
A Ali Hosseininaveh,10,1,-2.707129,0.000000,5,A Ali Hosseininaveh


In [16]:
df_author_score = df_author_score[['author_name', 'ranking_score_sum']]

In [17]:
df_author_score.head()

,author_name,ranking_score_sum
authors,,
(TYPE=name) (SCHEME=Vancouver) Kahn Ce,(TYPE=name) (SCHEME=Vancouver) Kahn Ce,-8.008489
-Jr. Paulo Drews,-Jr. Paulo Drews,-0.952377
344ehůřek,344ehůřek,-1.167167
A Aart Blokhuis,A Aart Blokhuis,-2.580458
A Ali Hosseininaveh,A Ali Hosseininaveh,-2.707129


In [18]:
from  itertools import chain

df_prediction_flatten_authors = pd.DataFrame({
        "authors": list(chain.from_iterable(df_prediction.authors)),
        "id": np.repeat(df_prediction.id.values, df_prediction.authors.str.len()),
})

In [19]:
df_prediction_flatten_authors['author_name'] = df_prediction_flatten_authors['authors']

In [20]:
df_prediction_flatten_authors = pd.merge(df_prediction_flatten_authors, df_author_score[['author_name', 'ranking_score_sum']], how='left', on='author_name')


In [21]:
df_prediction_flatten_authors.head()

,authors,id,author_name,ranking_score_sum
0,Altaf Hossain,001c8744-73c4-4b04-9364-22d31a10dbf1,Altaf Hossain,-1.309793
1,Faisal Zaman,001c8744-73c4-4b04-9364-22d31a10dbf1,Faisal Zaman,-5.457671
2,Mohammed Nasser,001c8744-73c4-4b04-9364-22d31a10dbf1,Mohammed Nasser,-4.787061
3,M. Mufakhkharul Islam,001c8744-73c4-4b04-9364-22d31a10dbf1,M. Mufakhkharul Islam,-1.309793
4,Alvaro L. Islas,00bcf2d5-1592-46b0-81fd-933f90b5ecca,Alvaro L. Islas,-2.190841


In [22]:
df_prediction_flatten_authors = df_prediction_flatten_authors[['id','ranking_score_sum']]

In [23]:
df_prediction_flatten_authors = df_prediction_flatten_authors.groupby(['id']).mean()

In [24]:
df_prediction_flatten_authors['id'] = df_prediction_flatten_authors.index

In [25]:
df_prediction_flatten_authors.head()

,ranking_score_sum,id
id,,
000000b8-7f59-49ad-b9bc-e92aa858fc37,0.088551,000000b8-7f59-49ad-b9bc-e92aa858fc37
00001301-0e56-4c9d-94aa-f776580aed87,-9.675334,00001301-0e56-4c9d-94aa-f776580aed87
00001bf6-0c0c-4264-a81b-0f1ddab7a25a,27.409006,00001bf6-0c0c-4264-a81b-0f1ddab7a25a
00001fb9-b0eb-49e5-b893-eb9b47c78910,-1.695634,00001fb9-b0eb-49e5-b893-eb9b47c78910
00003883-8fc4-4902-898a-fb47f9a5da8a,-3.649218,00003883-8fc4-4902-898a-fb47f9a5da8a


In [26]:
df_prediction_flatten_authors['author_score'] = df_prediction_flatten_authors['ranking_score_sum']

In [27]:
df_prediction_train = pd.merge(df_prediction, df_prediction_flatten_authors[['id', 'author_score']], how='inner', on='id')


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
df_prediction_train.head()

,id,authors,author_count,sentiment,Topic,domain_citation_avg,year,time_factor,ranking_score,author_score
0,001c8744-73c4-4b04-9364-22d31a10dbf1,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",4,1,1,57.087180,2009,8.399536e-09,-1.309793,-3.216079
1,00bcf2d5-1592-46b0-81fd-933f90b5ecca,"[Alvaro L. Islas, Constance M. Schober]",2,0,8,54.804301,2002,5.500736e-09,-1.254513,-2.190841
2,00c85316-bddf-4bcb-93f5-097adadd73c2,"[Patrick Cousot, Radhia Cousot]",2,1,2,55.424089,1991,6.520059e-08,-2.215803,38.787464
3,010d4ce9-0279-4166-ae73-14551ded6404,"[John R. Douceur, Jeremy Elson, Jon Howell, Ja...",4,1,9,52.136057,2008,4.063689e-09,16.544018,130.063854
4,012b88ae-a763-45d6-8f19-2ec9ff739d5f,"[Carmen Fernández-Daza Álvarez, Philippe Langl...",3,-1,5,55.809836,2004,6.345459e-09,-1.691160,-13.671207


In [ ]:
df_prediction_train.to_pickle ('prediction_training_data')